In [13]:
#packages
import pandas as pd
import pickle
import csv
import os
import re
import emoji
from textblob import TextBlob
import nltk
from langdetect import detect
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix
import gensim
nltk.download('vader_lexicon')
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import coo_matrix, hstack
from scipy.sparse import csr_matrix
import sklearn
from scipy import spatial
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegressionCV
import gensim.downloader
import fasttext
import csv
import nltk
nltk.download('stopwords')
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import random
import keras
import pickle
# import keras_metrics

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Import data

In [30]:
#import cleaded tweet
with open("./cleaned_strip@.pkl", 'rb') as f:
    clean_df = pickle.load(f)
clean_df["combined_tweet_txt"] = clean_df["tweet_txt"] + clean_df["og_tweet_txt"]

In [28]:
#import labeled training set
file_list = ["./sample_again.csv","./full_tweet_sample.csv"]
li = []
senti_df1 = pd.read_csv("./sample_again.csv").dropna()
senti_df1 = senti_df1[["full_txt", "sentiment"]]
senti_df2 = pd.read_csv("./full_tweet_sample.csv").dropna()
senti_df2 = senti_df2[["full_text", "sentiment"]]
senti_df2.columns = ["full_txt", "sentiment"]

df = pd.concat([senti_df1,senti_df2], axis=0, ignore_index=True)
X = df["full_txt"].tolist()
y = df["sentiment"].tolist()

tokens = [nltk.word_tokenize(sentences) for sentences in X]
# list_of_txt = list(sentiment_df["full_text"])
# true_sent = list(sentiment_df["sentiment"])
# sid = SentimentIntensityAnalyzer()

In [6]:
STOPWORDS = set(stopwords.words('english'))
random.seed(9)

In [29]:
## clean stopwords for training set 
special_word = ['rt','co','amp']
no_stop_X = []
for each in X:
    for word in STOPWORDS:
        token = ' ' + word + ' '
        each = each.replace(token, ' ')
        for spw in special_word:
            each = each.replace(spw, '')
        each = each.strip()
    no_stop_X.append(each)

In [31]:
## clean stopwords for data to label 
special_word = ['rt','co','amp']
# no_stop_X = []
def clean_stop(each):
    for word in STOPWORDS:
        token = ' ' + word + ' '
        each = each.replace(token, ' ')
        for spw in special_word:
            each = each.replace(spw, '')
        each = each.strip()
    return each
clean_df["combined_tweet_txt"] = clean_df["combined_tweet_txt"].apply(lambda x: clean_stop(x))

In [32]:
##split into train/test set
def split_set(no_stop_X, y):
    y_len = len(y)
    ratio = 0.8
    train_1_num = np.floor(y.count(1)*ratio)
    train_1_indices = [i for i, k in enumerate(y) if k == 1]
    train_neg_1_num = np.floor(y.count(-1)*ratio)
    train_neg_1_indices = [i for i, k in enumerate(y) if k == -1]
    train_0_num = np.floor(y.count(0)*ratio)
    train_0_indices = [i for i, k in enumerate(y) if k == 0]
    # random.selct
    train_1_set = random.sample(train_1_indices, int(train_1_num))
    train_neg_1_set = random.sample(train_neg_1_indices, int(train_neg_1_num))
    train_0_set = random.sample(train_0_indices, int(train_0_num))
    test_1_set = [i for i in train_1_indices if i not in train_1_set]
    test_neg_1_set = [i for i in train_neg_1_indices if i not in train_neg_1_set]
    test_0_set = [i for i in train_0_indices if i not in train_0_set]
    train_index = train_1_set + train_neg_1_set + train_0_set
    test_index = test_1_set + test_neg_1_set + test_0_set
    train_x = [no_stop_X[i] for i in train_index]
    train_y = [y[i] for i in train_index]
    test_x = [no_stop_X[i] for i in test_index]
    test_y = [y[i] for i in test_index]
#     print(train_y)
    return train_x, train_y, test_x, test_y

In [33]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 5000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 40
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
# clean_df["combined_tweet_txt"]
tokenizer.fit_on_texts(no_stop_X)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 5483 unique tokens.


In [35]:
X_total = tokenizer.texts_to_sequences(no_stop_X)
X_total = pad_sequences(X_total, maxlen=MAX_SEQUENCE_LENGTH)
max_list = []
# no_stop_X[0]
for each in no_stop_X:
    max_list.append(len(each.split()))
max(max_list)

85

In [41]:
## tokenize train x and y:
def tokenize(train_x,train_y, test_x, test_y, tokenizer):
    X_train_tokenize = tokenizer.texts_to_sequences(train_x)
    X_train_tokenize = pad_sequences(X_train_tokenize, maxlen=MAX_SEQUENCE_LENGTH)
    X_test_tokenize = tokenizer.texts_to_sequences(test_x)
    X_test_tokenize = pad_sequences(X_test_tokenize, maxlen=MAX_SEQUENCE_LENGTH)
    print('Shape of data tensor:', X_train_tokenize.shape)
    y_train_dummy = pd.get_dummies(train_y).values
    y_test_dummy = pd.get_dummies(test_y).values
    print('Shape of label tensor:', y_train_dummy.shape)
    return X_train_tokenize, y_train_dummy, X_test_tokenize, y_test_dummy

def lstm(MAX_NB_WORDS, EMBEDDING_DIM, X_train_tokenize, y_train_dummy, X_total):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_total.shape[1]))
    model.add(tf.keras.layers.SpatialDropout1D(0.2))
    model.add(tf.keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

    epochs = 6
    batch_size = 64

    history = model.fit(X_train_tokenize, y_train_dummy, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
    return model

def confusion(model,X_test_tokenize):
    labels = [-1, 0, 1]
    predictions = model.predict(X_test_tokenize)
    final_pred = [labels[np.argmax(pred)] for pred in predictions]
    a = confusion_matrix(test_y, final_pred, labels = labels)
    conf = pd.DataFrame(a, index=labels, columns=labels)
    recall = conf.loc[-1,-1]/(conf.loc[-1,-1] + conf.loc[-1,0] + conf.loc[-1,1])
    pres = conf.loc[-1,-1]/(conf.loc[-1,-1] + conf.loc[0,-1] + conf.loc[1,-1])
    f1 = 2*(recall*pres)/(recall+pres)
    return recall, pres, conf, f1
# predictions

In [48]:
#training lstm
from statistics import mean
cycle = range(5)
recall_list = []
pres_list = []
f1_list = []

for each in cycle:
    train_x, train_y, test_x, test_y = split_set(no_stop_X, y)
    X_train_tokenize, y_train_dummy, X_test_tokenize, y_test_dummy = tokenize(train_x,train_y, test_x, test_y, tokenizer)
    model = lstm(MAX_NB_WORDS, EMBEDDING_DIM, X_train_tokenize, y_train_dummy,  X_total)
    recall, pres, conf, f1 = confusion(model,X_test_tokenize)
    recall_list.append(recall)
    pres_list.append(pres)
    f1_list.append(f1)
    print(conf)

Shape of data tensor: (1597, 40)
Shape of label tensor: (1597, 3)
Epoch 1/6
23/23 [==============================] - 2s 83ms/step - loss: 0.9315 - precision_11: 0.6780 - recall_11: 0.4015 - val_loss: 0.2454 - val_precision_11: 1.0000 - val_recall_11: 1.0000
Epoch 2/6
23/23 [==============================] - 1s 62ms/step - loss: 0.8349 - precision_11: 0.6660 - recall_11: 0.6660 - val_loss: 0.3888 - val_precision_11: 1.0000 - val_recall_11: 1.0000
Epoch 3/6
23/23 [==============================] - 1s 61ms/step - loss: 0.7670 - precision_11: 0.6793 - recall_11: 0.6618 - val_loss: 0.2953 - val_precision_11: 1.0000 - val_recall_11: 0.9812
Epoch 4/6
23/23 [==============================] - 2s 65ms/step - loss: 0.6323 - precision_11: 0.7973 - recall_11: 0.6458 - val_loss: 0.4152 - val_precision_11: 0.9786 - val_recall_11: 0.8562
    -1    0   1
-1  27   59   0
 0  18  262   0
 1   4   31   0
Shape of data tensor: (1597, 40)
Shape of label tensor: (1597, 3)
Epoch 1/6
23/23 [===================

In [46]:
recall_list

[0.3372093023255814,
 0.5465116279069767,
 0.5232558139534884,
 0.46511627906976744,
 0.43023255813953487]

In [47]:
recall_mean = mean(recall_list)
pres_mean = mean(pres_list)
print("At epoch = 6, and -1 classification, avg pres is",pres_mean,"and avg recall is",recall_mean)

At epoch = 6, and -1 classification, avg pres is 0.41730363263850245 and avg recall is 0.4604651162790698


In [49]:
accr = model.evaluate(X_test_tokenize,y_test_dummy)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f} Recall: {:0.3f}'.format(accr[0],accr[1],accr[2]))

13/13 [==============================] - 0s 8ms/step - loss: 1.0260 - precision_15: 0.7258 - recall_15: 0.6534
Test set
  Loss: 1.026
  Accuracy: 0.726 Recall: 0.653


# Generate classification for the whole set using LSTM model

In [50]:
def predict_whole(txt_input):
    new_complaint = [txt_input]
    seq = tokenizer.texts_to_sequences(new_complaint)
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred = model.predict(padded)
    labels = [-1, 0, 1]
    return labels[np.argmax(pred)]
clean_df["prediction"] = clean_df["combined_tweet_txt"].apply(lambda x: predict_whole(x))

In [51]:
saving_df = clean_df[["tweet_id", "og_tweet_id", "combined_tweet_txt", "user_loc","prediction"]]
saving_df.to_csv ('./labled_whole_set.csv', index = False, header=True,float_format='{:f}'.format)

In [63]:
no_retweet = clean_df[clean_df['og_tweet_time'].isna()].copy()
retweet = clean_df[clean_df['og_tweet_time'].notna()].copy()

In [75]:
len(retweet.loc[retweet['prediction'] == 1])/len(retweet)

0.8841226278891302

In [76]:
len(retweet.loc[retweet['prediction'] == 1])

203

In [66]:
no_retweet

,og_tweet_txt,og_tweet_time,og_tweet_id,og_tweet_user_id,og_tweet_user_name,og_tweet_user_desc,og_tweet_user_vrifd,og_tweet_user_loc,tweet_txt,user_loc,...,user_name,tweet_likes,tweet_source,hashtags,user_acc_cr_time,user_verified,user_total_tweets,user_followers,combined_tweet_txt,prediction
3,,NaT,,,,,,,is anyone gonna mention biden forgot he receiv...,On a Lake,...,tinberry,0,Twitter Web App,,2009-04-23 21:51:32,False,1737,21,is anyone gonna mention biden forgot received ...,0
7,,NaT,,,,,,,just on presentation alone he should be put in...,"Saratoga Springs, NY",...,SML531974,0,Twitter for Android,,2020-01-17 12:41:56,False,13152,452,just presentation alone put rner,0
9,,NaT,,,,,,,claire pfizer only enrolled 5 ppl over the age...,World,...,CarmenNasty2016,0,Twitter Web App,,2016-11-11 02:53:56,False,2238,39,claire pfizer enrolled 5 ppl age ppl vaccinate...,0
10,,NaT,,,,,,,is this biden s uniqua,None,...,BotSatire,0,Twitter Web App,,2020-08-07 03:13:40,False,1185,153,is biden uniqua,0
18,,NaT,,,,,,,a coronavirus vaccine nears final tests in cub...,None,...,newspointpn,0,IFTTT,,2020-04-03 06:34:21,False,4977,2,a ronavirus vaccine nears final tests cuba tou...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970674,,NaT,,,,,,,pushpin ghana covid 19 vaccine timelines,Accra,...,OnuaTV,0,Twitter for iPhone,OnuaMaakye,2019-07-26 10:35:15,False,7451,2569,pushpin ghana vid 19 vaccine timelines,0
970682,,NaT,,,,,,,you know the covid vaccine doesn t work what p...,None,...,banthebbc,0,Twitter Web App,,2012-12-25 19:05:38,False,98877,81975,you know vid vaccine work possible reason uld ...,0
970683,,NaT,,,,,,,israeli researchers find covid vaccine working...,None,...,joseffederman,0,Twitter for iPhone,,2011-02-09 09:40:45,False,1662,2274,israeli researchers find vid vaccine working p...,0
970687,,NaT,,,,,,,nana addo to take the first shot of the covid ...,"Accra, Ghana",...,sabonzy,0,Twitter for Android,,2012-01-17 22:53:01,False,42764,16715,nana addo take first shot vid 19 vaccine broug...,0
